In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [3]:
folder = os.listdir()

In [296]:
file = []
for f in folder:
    if "실거래가" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
gajwa = gajwa.dropna(axis=0)

In [297]:
rate = pd.read_csv("./지역별_전월세전환율_20221112193907.csv", encoding="cp949")
rate = rate.T
rate = rate[2:]

# 계약년월 기준으로 전월세전환율 입력하기
month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep

# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa["월세(만원)"][i] * 12 / (gajwa["전월세전환율"][i]/100) + gajwa["보증금(만원)"][i]) for i in range(len(gajwa))]
gajwa["value"] = value

gajwa

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,전월세전환율,value
0,경상남도 진주시 가좌동,1***,12m미만,15.0,월세,202107,4,150,30,2016.0,가좌길48번길,6.4,5775
1,경상남도 진주시 가좌동,1***,12m미만,19.0,월세,202107,17,200,30,2019.0,가좌길48번길,6.4,5825
2,경상남도 진주시 가좌동,1***,12m미만,19.0,월세,202107,18,5000,10,2019.0,가좌길48번길,6.4,6875
3,경상남도 진주시 가좌동,1***,8m미만,19.8,월세,202107,17,200,32,2017.0,가좌길60번길,6.4,6200
4,경상남도 진주시 가좌동,1***,12m미만,20.0,월세,202107,21,100,25,2015.0,가좌길48번길,6.4,4788
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,경상남도 진주시 가좌동,1***,8m미만,30.0,전세,201910,20,6000,0,2018.0,가좌길44번길,7.0,6000
1615,경상남도 진주시 가좌동,4**,12m미만,33.0,월세,201910,20,1500,30,2010.0,개양로6번길,7.0,6643
1616,경상남도 진주시 가좌동,5**,12m미만,25.0,월세,201904,5,300,33,2007.0,진주대로500번길,6.0,6900
1617,경상남도 진주시 가좌동,4**,12m미만,27.0,월세,201904,16,250,30,2007.0,개양로6번길,6.0,6250
